In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
import copy

# import yfinance as yf

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

# %load_ext autoreload
# %autoreload 2

In [3]:
mydateparser = lambda x: datetime.strptime(x, "%Y-%m-%d")

In [4]:
labels = pd.read_csv('labels_sector.csv', index_col='Date', parse_dates=True, date_parser=mydateparser)
labels.iloc[:15,:12]

,T,TWTR,FB,CMCSA,VZ,NFLX,DIS,ATVI,IPG,DISCA,F,GM
Date,,,,,,,,,,,,
2016-01-22,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2016-01-25,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2016-01-26,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-27,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-28,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-29,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-02-01,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-02-02,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2016-02-03,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0


In [5]:
# print(labels.iloc[:15,:12].to_latex(float_format="%.0f"))

In [6]:
stocks = pd.read_csv('stocks_by_cluster.csv')
stocks

,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8
0,PEP,TGT,ABT,MSFT,CVX,DVN,DAL,JPM
1,KO,TJX,PFE,AMAT,SLB,MRO,AAL,BAC
2,PG,WBA,MDT,AAPL,COP,KMI,UAL,MS
3,PM,WMT,ABBV,INTC,HAL,WMB,UNP,C
4,MDLZ,CVS,JNJ,NFLX,XOM,MGM,FAST,KEY
5,MO,M,BSX,NVDA,CAT,HPE,IPG,SCHW
6,NWL,KR,GILD,ATVI,RF,DISCA,GM,HBAN


In [11]:
print(stocks.to_latex(index=False))

\begin{tabular}{llllllll}
\toprule
cluster\_1 & cluster\_2 & cluster\_3 & cluster\_4 & cluster\_5 & cluster\_6 & cluster\_7 & cluster\_8 \\
\midrule
      PEP &       TGT &       ABT &      MSFT &       CVX &       DVN &       DAL &       JPM \\
       KO &       TJX &       PFE &      AMAT &       SLB &       MRO &       AAL &       BAC \\
       PG &       WBA &       MDT &      AAPL &       COP &       KMI &       UAL &        MS \\
       PM &       WMT &      ABBV &      INTC &       HAL &       WMB &       UNP &         C \\
     MDLZ &       CVS &       JNJ &      NFLX &       XOM &       MGM &      FAST &       KEY \\
       MO &         M &       BSX &      NVDA &       CAT &       HPE &       IPG &      SCHW \\
      NWL &        KR &      GILD &      ATVI &        RF &     DISCA &        GM &      HBAN \\
\bottomrule
\end{tabular}



In [20]:
stocks = stocks['Information Technology'].to_frame().loc[0].to_frame().T

In [21]:
from GRTEL.indicators import get_RSI, get_Stochastic_Oscillator, get_Williams, get_MACD, get_OBV

In [24]:
samples = []
labels = []
for sector in stocks.columns:
    for i, stock in enumerate(stocks[sector].values):
        
        stock_name = stock
        
        # get the original data
        data = yf.download(stock, start='2016-01-02', end='2019-01-01')

        # calculate change in price
        data['change_in_price'] = data['Close'].diff()

        # calculate indicators
        data['RSI'] = get_RSI(data)
        data['k_percent'] = get_Stochastic_Oscillator(data)
        data['r_percent'] = get_Williams(data)

        # Calculate the MACD
        macd, ema_9_macd = get_MACD(data)
        data['MACD'] = macd
        data['MACD_EMA9'] = ema_9_macd

        # Calculate the 9-day Price Rate of Change
        data['Price Rate Of Change'] = data['Close'].pct_change(periods=9)

        # Calculate On Balance Volume
        data['On Balance Volume'] = get_OBV(data)

        # Create the predicition column (To keep this as a binary classifier we'll consider flat days as up days)
        days_out = 9
        data['Prediction'] = np.sign(np.sign(data['Close'].shift(-days_out) - data['Close']) + 1.)

        # Drop rows with NaN.
#         data = data.dropna()

        X_i = data[['Close', 'RSI', 'k_percent', 'r_percent', 'MACD', 'MACD_EMA9', 'Price Rate Of Change', 'On Balance Volume']].copy()
        X_i.columns = [[sector]*len(X_i.columns), [stock_name]*len(X_i.columns), X_i.columns]
        
        y_i = data['Prediction'].copy()
        y_i.name = stock
        
        samples.append(X_i)
        labels.append(y_i)

        
samples = pd.concat(samples, axis=1)
samples.columns.names = ['Sector', 'Asset', 'Metrics']

labels = pd.concat(labels, axis=1)

[*********************100%***********************]  1 of 1 completed


In [28]:
samples[:16]

Sector     Information Technology                                             \
Asset                        AAPL                                              
Metrics                     Close        RSI  k_percent  r_percent      MACD   
Date                                                                           
2016-01-04              26.337500        NaN        NaN        NaN  0.000000   
2016-01-05              25.677500   0.000000        NaN        NaN -0.014808   
2016-01-06              25.174999   0.000000        NaN        NaN -0.034450   
2016-01-07              24.112499   0.000000        NaN        NaN -0.080901   
2016-01-08              24.240000   6.871851        NaN        NaN -0.099365   
2016-01-11              24.632500  25.143533        NaN        NaN -0.091813   
2016-01-12              24.990000  37.940091        NaN        NaN -0.069193   
2016-01-13              24.347500  28.010531        NaN        NaN -0.083471   
2016-01-14              24.879999  42.421306        NaN        NaN -0.065668   
2016-01-15              24.282499  33.689891        NaN        NaN -0.082861   
2016-01-19              24.165001  32.186684        NaN        NaN -0.100382   
2016-01-20              24.197500  33.138786        NaN        NaN -0.110077   
2016-01-21              24.075001  31.231692        NaN        NaN -0.122727   
2016-01-22              25.355000  59.400922  64.360417 -35.639583 -0.056418   
2016-01-25              24.860001  50.221436  48.431248 -51.568752 -0.035065   
2016-01-26              24.997499  52.570611  73.407780 -26.592220 -0.010082   

Sector                                                       
Asset                                                        
Metrics    MACD_EMA9 Price Rate Of Change On Balance Volume  
Date                                                         
2016-01-04  0.000000                  NaN                 0  
2016-01-05 -0.008226                  NaN        -223164000  
2016-01-06 -0.018974                  NaN        -496993600  
2016-01-07 -0.039952                  NaN        -821371200  
2016-01-08 -0.057626                  NaN        -538179200  
2016-01-11 -0.066893                  NaN        -339221600  
2016-01-12 -0.067475                  NaN        -142604800  
2016-01-13 -0.071319                  NaN        -392363200  
2016-01-14 -0.070014                  NaN        -139682800  
2016-01-15 -0.072892            -0.078026        -459018400  
2016-01-19 -0.078907            -0.058904        -671369200  
2016-01-20 -0.085601            -0.038828        -382031600  
2016-01-21 -0.093458            -0.001555        -590677600  
2016-01-22 -0.085709             0.045998        -327475600  
2016-01-25 -0.075211             0.009236        -534653600  
2016-01-26 -0.061808             0.000300        -234345600

In [32]:
print(samples[:16].to_latex(float_format="%.2f"))

\begin{tabular}{lrrrrrrrr}
\toprule
Sector & \multicolumn{8}{l}{Information Technology} \\
Asset & \multicolumn{8}{l}{AAPL} \\
Metrics &                  Close &   RSI & k\_percent & r\_percent &  MACD & MACD\_EMA9 & Price Rate Of Change & On Balance Volume \\
Date       &                        &       &           &           &       &           &                      &                   \\
\midrule
2016-01-04 &                  26.34 &   NaN &       NaN &       NaN &  0.00 &      0.00 &                  NaN &                 0 \\
2016-01-05 &                  25.68 &  0.00 &       NaN &       NaN & -0.01 &     -0.01 &                  NaN &        -223164000 \\
2016-01-06 &                  25.17 &  0.00 &       NaN &       NaN & -0.03 &     -0.02 &                  NaN &        -496993600 \\
2016-01-07 &                  24.11 &  0.00 &       NaN &       NaN & -0.08 &     -0.04 &                  NaN &        -821371200 \\
2016-01-08 &                  24.24 &  6.87 &       NaN &   